# BASIC VALIDATION NOTEBOOK

This is a basic validation notebook for static energy calculation with the magnetic ACE potential.

It requires the following external python packages:

- Atomic Simulation Environment (ASE): 
https://wiki.fysik.dtu.dk/ase/index.html

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load ASE (REQUIRED)
from ase.calculators.lammpsrun import LAMMPS
from ase.constraints import StrainFilter
from ase.build import bulk
from ase import Atoms

# Load standard python modules
import shutil
import os
import glob
import numpy as np
import copy
import matplotlib.pyplot as plt
import random
import math

# USER INPUT
- define the calculator and its parameters
- name, mass and equilibrium crystal structure of the calculated element

In [3]:
!pwd

/home/utente/Documenti/fitFe_tesi/fortran_code_for_submission/ace_etot_submission/example


In [4]:
# Save starting directory
save_cwd=os.getcwd()
print("Current directory: ",save_cwd)

# Set where lammps calculations will be done
tmp_folder="/tmp/test_ace"

# Better remove whatever is left there
shutil.rmtree(tmp_folder, ignore_errors = True)
print("Calculations will run in directory: ",tmp_folder)
if not os.path.isdir(tmp_folder):
    os.mkdir(tmp_folder)

Current directory:  /home/utente/Documenti/fitFe_tesi/fortran_code_for_submission/ace_etot_submission/example
Calculations will run in directory:  /tmp/test_ace


In [5]:
path_to_potential = '/home/utente/lammps-ace/examples/SPIN/iron_test_s'
path_to_binary = '/home/utente/Documenti/lammps-ace_etot1/src'

In [6]:
# ACE parameter file

model = 'pot_Fe_empty.ace'
potential='{}/pot_Fe_empty.ace'.format(path_to_potential)

In [7]:
# LAMMPS binary

os.environ['ASE_LAMMPSRUN_COMMAND']='{}/lmp_serial'.format(path_to_binary)

In [8]:
# Set up LAMMPS calculator in ASE

parameters = {'pair_style':'hybrid/overlay pace spin/ace 1 4.5 4.0 pbe 0.2',
              'pair_coeff':['* * pace {} Fe'.format(potential),'* * spin/ace ace Fe'],
              'tmp_dir':tmp_folder,
              'atom_style':'spin'}

lammps = LAMMPS(**parameters)

In [9]:
os.system('cp {}/Fe.pbe.in {}'.format(path_to_potential, tmp_folder))

0

# STATIC ENERGY CALCULATION

In [10]:
latpar = 2.83

In [11]:
m_eq = 2.22

In [12]:
unitcell = bulk("Fe",crystalstructure="bcc",a=latpar,cubic=True)

In [13]:
mag_mom = np.array([[0., 0., m_eq], [0., 0., m_eq]])
unitcell.set_initial_magnetic_moments(mag_mom)

In [14]:
unitcell.set_calculator(lammps)

In [15]:
unitcell.get_total_energy()

-15.7479878560111

In [16]:
unitcell.get_forces()

array([[0.00000000e+00, 1.38777878e-17, 3.36856679e-17],
       [5.55111512e-17, 4.51028104e-17, 9.65858078e-17]])